In [1]:
import sys
sys.path.append(str('/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/'))
import pandas as pd
from configparser import ConfigParser

from new_repo.MyConfigParser import MyConfigParser
from new_repo.LoadData import DataLoader
from new_repo.QueryingData import Combine
from new_repo.FileGenerator import *

import new_repo.Accuracy
import new_repo.BuildModel
import new_repo.Optimization 


In [2]:

config_in_path = "/home/jupyter/GroupM_US_Offshore_Hub_Savant/All Regions/Device type/it/24 Sep Update/config ups IT 24 Sep.ini"
config_file = MyConfigParser(config_in_path)
config_dict = config_file.GetDict()

region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]

base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = base_path + config_dict["FILEPATH"]["constraint_file_name"]
breakdown_ls = [config_dict['CLIENTINFO'][i] for i in [f'breakdown{i}' for i in range(1,5)] if config_dict['CLIENTINFO'][i] != None]

small_market_str = ''
pre_forecast_period_budget = float(config_dict['CLIENTINFO']['pre_forecast_period_budget'])
client_budget = float(config_dict['CLIENTINFO']['client_budget'])

save_date = config_dict['CLIENTINFO']['save_date']

In [3]:
# Get forecasters
bdDf = pd.read_csv(cuts_df_in_path)#.drop('Unnamed: 0', axis = 1)
cuts_df = bdDf.reset_index(drop = True)
cuts_df['forecaster_name_list'] =  cuts_df['market'] + ' ' + cuts_df['account_type'] + ' ' + cuts_df['funnel'] + '.pkl' #' 05.19.2021 Final.pkl'
cuts_df['periods'] = periods
forecasters_array = cuts_df.forecaster_name_list

# Get analytical file
analytical_file = pd.read_csv(analytical_file_path).drop('Unnamed: 0', axis = 1)
analytical_file['ds'] = pd.to_datetime(analytical_file['ds'])
analytical_file_check_version = analytical_file.copy()

# Get constrain file
constraint_file = pd.read_csv(constraint_file_name)

In [5]:
today = '06.05.2024' #datetime.date.today().strftime("%m.%d.%Y")# 
fileName = base_path + f'Pos Result {today}.csv' #today
pos_df = pd.read_csv(fileName).drop('Unnamed: 0', axis = 1)
# pos_df = pd.DataFrame({'variable_splits':analytical_file.columns[19:37],'pos': 1})
# pos_df.pos = 1
pos_df

,variable_splits,pos
0,Italy_Desktop_-_Apr_2024,0.931793
1,Italy_Desktop_-_May_2024,1.140198
2,Italy_Desktop_-_Jun_2024,1.138830
3,Italy_Desktop_-_Jul_2024,1.189274
4,Italy_Desktop_-_Aug_2024,1.190126
5,Italy_Desktop_-_Sep_2024,1.124223
6,Italy_Desktop_-_Oct_2024,1.183638
7,Italy_Desktop_-_Nov_2024,0.927759
8,Italy_Desktop_-_Dec_2024,0.854373
9,Italy_Mobile_&_Tablet_-_Apr_2024,1.027967


In [6]:
resultPSO = new_repo.Optimization.SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights = pos_df.pos.to_numpy()

daily_simulated_forecast = resultPSO.ResultGenerator(inputWeights)

[0.93179303 1.14019776 1.13882997 1.18927369 1.19012578 1.12422273
 1.18363847 0.92775924 0.85437316 1.02796697 0.80089549 0.80081357
 0.96949578 1.12118105 0.81248777 0.92303019 0.94417229 0.83718183]
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  359972.0 difference between planned and forecast X:  0.0
after adjust the weight, the previous period period spend is: 59068.224999999984
yhat: 58241.0
analytical_spend_latest_period: 300904.0300000001
client_budget - analytical_spend_latest_period: 0


In [7]:
resultPSO = new_repo.Optimization.SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights_base = np.ones(constraint_file.shape[0])

daily_simulated_forecast_base = resultPSO.ResultGenerator(inputWeights_base)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  359972.0 difference between planned and forecast X:  0.0
after adjust the weight, the previous period period spend is: 59068.22499999999
yhat: 56800.0
analytical_spend_latest_period: 300904.02999999997
client_budget - analytical_spend_latest_period: 0


In [8]:
forecasted_daily_df = daily_simulated_forecast[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
forecasted_daily_df['month'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
forecasted_daily_df['year'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%Y').astype(str)

forecasted_daily_df = forecasted_daily_df.rename(columns = { 'yhat': 'OptimalConversionQuantity', 'X': 'OptimalSpendValue'})
forecasted_daily_df['OptimalSpendValue'] = forecasted_daily_df['OptimalSpendValue'].round(2)

forecasted_daily_df['variable_splits'] = forecasted_daily_df['market'] + '_' + forecasted_daily_df['account_type']+'_'+forecasted_daily_df['funnel']+'_'+forecasted_daily_df['month']+'_'+forecasted_daily_df['year']
print(forecasted_daily_df.OptimalSpendValue.sum(), forecasted_daily_df.OptimalConversionQuantity.sum())
forecasted_daily_df

359972.16000000003 58241.0


,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits
0,Italy,Desktop,-,2024-04-25,474.0,2667.25,Apr,2024,Italy_Desktop_-_Apr_2024
1,Italy,Desktop,-,2024-04-26,832.0,3832.71,Apr,2024,Italy_Desktop_-_Apr_2024
2,Italy,Desktop,-,2024-04-27,101.0,464.97,Apr,2024,Italy_Desktop_-_Apr_2024
3,Italy,Desktop,-,2024-04-28,66.0,304.85,Apr,2024,Italy_Desktop_-_Apr_2024
4,Italy,Desktop,-,2024-04-29,178.0,823.11,Apr,2024,Italy_Desktop_-_Apr_2024
...,...,...,...,...,...,...,...,...,...
497,Italy,Mobile & Tablet,-,2024-12-27,18.0,147.94,Dec,2024,Italy_Mobile & Tablet_-_Dec_2024
498,Italy,Mobile & Tablet,-,2024-12-28,12.0,147.94,Dec,2024,Italy_Mobile & Tablet_-_Dec_2024
499,Italy,Mobile & Tablet,-,2024-12-29,15.0,147.94,Dec,2024,Italy_Mobile & Tablet_-_Dec_2024
500,Italy,Mobile & Tablet,-,2024-12-30,25.0,54.92,Dec,2024,Italy_Mobile & Tablet_-_Dec_2024


In [9]:
base_daily_df = daily_simulated_forecast_base[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
base_daily_df['month'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
base_daily_df['year'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%Y').astype(str)
base_daily_df['X'] = base_daily_df['X'].round(2)
base_daily_df = base_daily_df.rename(columns = { 'X': 'BaseSpend', 'yhat': 'BaseConversionQuantity'})

print(base_daily_df.BaseSpend.sum(), base_daily_df.BaseConversionQuantity.sum())
base_daily_df

359972.24 56800.0


,market,account_type,funnel,ds,BaseConversionQuantity,BaseSpend,month,year
0,Italy,Desktop,-,2024-04-25,561.0,3067.80,Apr,2024
1,Italy,Desktop,-,2024-04-26,957.0,4408.27,Apr,2024
2,Italy,Desktop,-,2024-04-27,116.0,534.79,Apr,2024
3,Italy,Desktop,-,2024-04-28,76.0,350.63,Apr,2024
4,Italy,Desktop,-,2024-04-29,205.0,946.72,Apr,2024
...,...,...,...,...,...,...,...,...
497,Italy,Mobile & Tablet,-,2024-12-27,19.0,185.21,Dec,2024
498,Italy,Mobile & Tablet,-,2024-12-28,13.0,185.21,Dec,2024
499,Italy,Mobile & Tablet,-,2024-12-29,16.0,185.21,Dec,2024
500,Italy,Mobile & Tablet,-,2024-12-30,25.0,68.75,Dec,2024


In [10]:
forecast_n_base_df = pd.merge(forecasted_daily_df, base_daily_df, how = 'left', on = ['market', 'account_type', 'funnel', 'ds', 'month', 'year'])
forecast_n_base_df['SavedDate'] = datetime.date.today().strftime("%Y-%m-%d")
forecast_n_base_df = forecast_n_base_df.rename(columns=lambda x: x[0:1].upper() + x[1:])
forecast_n_base_df = forecast_n_base_df.drop(['Variable_splits'], axis = 1)
# forecast_n_base_df.insert(3, 'WeekStart', [ i for i in forecast_n_base_df.Ds.apply(lambda x: x - pd.offsets.Week(weekday=4) )])
forecast_n_base_df.insert(0, 'Region', region)

forecast_n_base_df[forecast_n_base_df.Ds>='2024-01-01'].sum(numeric_only=1)

OptimalConversionQuantity     58241.00
OptimalSpendValue            359972.16
BaseConversionQuantity        56800.00
BaseSpend                    359972.24
dtype: float64

In [11]:
daily_simulated_forecast[((daily_simulated_forecast['account_type'] == 'Desktop')
                   | (daily_simulated_forecast['account_type'] == 'Mobile & Tablet'))]

,region,market,funnel,account_type,ds,X,Month,Year,Italy_Desktop_-_Apr_2024,Italy_Desktop_-_Aug_2024,...,Italy_Mobile_&_Tablet_-_Jul_2024,Italy_Mobile_&_Tablet_-_Jun_2024,Italy_Mobile_&_Tablet_-_May_2024,Italy_Mobile_&_Tablet_-_Nov_2024,Italy_Mobile_&_Tablet_-_Oct_2024,Italy_Mobile_&_Tablet_-_Sep_2024,Weekend,AOV,ROAS,yhat
0,Europe,Italy,-,Desktop,2024-04-25,2667.249936,Apr,2024,2667.249936,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,38.5,1.90,474.0
1,Europe,Italy,-,Desktop,2024-04-26,3832.709629,Apr,2024,3832.709629,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,38.5,1.90,832.0
2,Europe,Italy,-,Desktop,2024-04-27,464.966062,Apr,2024,464.966062,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,38.5,1.90,101.0
3,Europe,Italy,-,Desktop,2024-04-28,304.847606,Apr,2024,304.847606,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,38.5,1.90,66.0
4,Europe,Italy,-,Desktop,2024-04-29,823.108227,Apr,2024,823.108227,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,38.5,1.90,178.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,Europe,Italy,-,Mobile & Tablet,2024-12-27,147.944264,Dec,2024,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,17.3,0.33,18.0
498,Europe,Italy,-,Mobile & Tablet,2024-12-28,147.944264,Dec,2024,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,17.3,0.33,12.0
499,Europe,Italy,-,Mobile & Tablet,2024-12-29,147.944264,Dec,2024,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,17.3,0.33,15.0
500,Europe,Italy,-,Mobile & Tablet,2024-12-30,54.916700,Dec,2024,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,17.3,0.33,25.0


In [12]:
forecast_n_base_df.loc[((daily_simulated_forecast['account_type'] == 'Desktop')
                   | (daily_simulated_forecast['account_type'] == 'Mobile & Tablet'))]

,Region,Market,Account_type,Funnel,Ds,OptimalConversionQuantity,OptimalSpendValue,Month,Year,BaseConversionQuantity,BaseSpend,SavedDate
0,Italy,Italy,Desktop,-,2024-04-25,474.0,2667.25,Apr,2024,561.0,3067.80,2024-06-05
1,Italy,Italy,Desktop,-,2024-04-26,832.0,3832.71,Apr,2024,957.0,4408.27,2024-06-05
2,Italy,Italy,Desktop,-,2024-04-27,101.0,464.97,Apr,2024,116.0,534.79,2024-06-05
3,Italy,Italy,Desktop,-,2024-04-28,66.0,304.85,Apr,2024,76.0,350.63,2024-06-05
4,Italy,Italy,Desktop,-,2024-04-29,178.0,823.11,Apr,2024,205.0,946.72,2024-06-05
...,...,...,...,...,...,...,...,...,...,...,...,...
497,Italy,Italy,Mobile & Tablet,-,2024-12-27,18.0,147.94,Dec,2024,19.0,185.21,2024-06-05
498,Italy,Italy,Mobile & Tablet,-,2024-12-28,12.0,147.94,Dec,2024,13.0,185.21,2024-06-05
499,Italy,Italy,Mobile & Tablet,-,2024-12-29,15.0,147.94,Dec,2024,16.0,185.21,2024-06-05
500,Italy,Italy,Mobile & Tablet,-,2024-12-30,25.0,54.92,Dec,2024,25.0,68.75,2024-06-05


In [13]:
forecast_n_base_df = forecast_n_base_df.rename(columns={'Account_type': 'device_segment'})
forecast_n_base_df = forecast_n_base_df.drop('Funnel', axis=1)

In [14]:
forecast_n_base_df[['Year']] = forecast_n_base_df[['Year']].astype(int)
forecast_n_base_df["SavedDate"] = pd.to_datetime(forecast_n_base_df["SavedDate"])
forecast_n_base_df['SavedDate'] = forecast_n_base_df['SavedDate'].dt.date

In [15]:
# Construct a BigQuery client object.
bqclient = bigquery.Client()

target_table_name = 'cost_by_device_' + region + save_date
table_id = "ups-analytics.forecast_optimization." + target_table_name

# job_config.
job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")

load_job = bqclient.load_table_from_dataframe(
    forecast_n_base_df[forecast_n_base_df.Ds>='2024-06-01'], table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = bqclient.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 428 rows.


table = bqclient.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

In [16]:
forecast_n_base_df[forecast_n_base_df.Ds>='2024-06-01'].to_csv(base_path + 'IT Output.csv')